<a href="https://colab.research.google.com/github/as9786/ComputerVision/blob/main/model/code/SqueezeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module

In [1]:
import numpy as np
import tensorflow as tf

# Data

In [2]:
(X_train,y_train),(X_test,y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [3]:
X_train.shape

(60000, 28, 28)

# Data preprocessing

In [10]:
X_train = X_train.reshape(X_train.shape[0],28,28,1).astype('float32')
X_test = X_test.reshape(X_test.shape[0],28,28,1).astype('float32')

In [11]:
# Noramliazation
X_train /= 255.
X_test /= 255.

In [12]:
# one hot encoding
y_train_one_hot = tf.keras.utils.to_categorical(y_train)
y_test_one_hot = tf.keras.utils.to_categorical(y_test)

In [14]:
y_train_one_hot[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

# 모형

In [18]:
def fire_module(x, n_of_squeeze_layer,bypass=False):
  n_of_expand_layer = 4 * n_of_squeeze_layer
  squeeze = tf.keras.layers.Conv2D(n_of_squeeze_layer,(1,1),activation='relu',padding='same')(x)
  expand_1x1 = tf.keras.layers.Conv2D(n_of_expand_layer,(1,1),activation='relu',padding='same')(squeeze)
  expand_3x3 = tf.keras.layers.Conv2D(n_of_expand_layer,(3,3),activation='relu',padding='same')(squeeze)

  x_re = tf.keras.layers.Concatenate(axis=1)([expand_1x1,expand_3x3])

  if bypass:
    x_re = tf.keras.layers.Add()([x_re,x])

  return x_re

In [24]:
# SqueezeNet
input = tf.keras.layers.Input(shape=(28,28,1))
x = tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu',strides=(2,2),padding='same')(input)
x = tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)

x = fire_module(x,16)
x = fire_module(x,16)
x = tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)

x = fire_module(x,64)
x = fire_module(x,64)

# classification
x = tf.keras.layers.Conv2D(10,(1,1),strides=(1,1))(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Activation('softmax')(x)

model = tf.keras.models.Model(input,output)

In [25]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_34 (Conv2D)             (None, 14, 14, 64)   640         ['input_4[0][0]']                
                                                                                                  
 max_pooling2d_6 (MaxPooling2D)  (None, 6, 6, 64)    0           ['conv2d_34[0][0]']              
                                                                                                  
 conv2d_35 (Conv2D)             (None, 6, 6, 16)     1040        ['max_pooling2d_6[0][0]']        
                                                                                            

# 학습

In [28]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [29]:
model.fit(X_train,y_train_one_hot,epochs=50,batch_size=128,validation_split=0.2)

Epoch 1/50
375/375 [==============================] - 5s 12ms/step - loss: 1.1486 - accuracy: 0.5699 - val_loss: 0.4519 - val_accuracy: 0.8442
Epoch 2/50
375/375 [==============================] - 4s 11ms/step - loss: 0.2664 - accuracy: 0.9158 - val_loss: 0.1383 - val_accuracy: 0.9573
Epoch 3/50
375/375 [==============================] - 4s 11ms/step - loss: 0.1403 - accuracy: 0.9568 - val_loss: 0.0940 - val_accuracy: 0.9706
Epoch 4/50
375/375 [==============================] - 4s 12ms/step - loss: 0.1011 - accuracy: 0.9687 - val_loss: 0.1049 - val_accuracy: 0.9698
Epoch 5/50
375/375 [==============================] - 4s 11ms/step - loss: 0.0839 - accuracy: 0.9737 - val_loss: 0.0679 - val_accuracy: 0.9793
Epoch 6/50
375/375 [==============================] - 4s 12ms/step - loss: 0.0675 - accuracy: 0.9792 - val_loss: 0.0629 - val_accuracy: 0.9784
Epoch 7/50
375/375 [==============================] - 4s 12ms/step - loss: 0.0610 - accuracy: 0.9811 - val_loss: 0.0785 - val_accuracy: 0.9758

# 평가

In [30]:
model.evaluate(X_test,y_test_one_hot,verbose=0)

[0.04331129044294357, 0.9904000163078308]